# Testando DB

In [1]:
!pip install langchain_community

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [2]:
!pip install -U transformers accelerate torch

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [3]:
!pip install -U bitsandbytes


/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)


In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///sus_data.db")
nome_tabela = "dados_sus3"


In [5]:
# informacao_tabela = db.get_table_info([nome_tabela])
print(db.get_table_info([nome_tabela]))


CREATE TABLE dados_sus3 (
	"DIAG_PRINC" TEXT, 
	"MUNIC_RES" BIGINT, 
	"MUNIC_MOV" BIGINT, 
	"PROC_REA" BIGINT, 
	"IDADE" BIGINT, 
	"SEXO" BIGINT, 
	"CID_MORTE" TEXT, 
	"MORTE" BIGINT, 
	"CNES" BIGINT, 
	"VAL_TOT" FLOAT, 
	"UTI_MES_TO" BIGINT, 
	"DT_INTER" BIGINT, 
	"DT_SAIDA" BIGINT, 
	total_ocorrencias BIGINT, 
	"UF_RESIDENCIA_PACIENTE" TEXT, 
	"CIDADE_RESIDENCIA_PACIENTE" TEXT, 
	"LATI_CIDADE_RES" FLOAT, 
	"LONG_CIDADE_RES" FLOAT
)

/*
3 rows from dados_sus3 table:
DIAG_PRINC	MUNIC_RES	MUNIC_MOV	PROC_REA	IDADE	SEXO	CID_MORTE	MORTE	CNES	VAL_TOT	UTI_MES_TO	DT_INTER	DT_SAIDA	total_ocorrencias	UF_RESIDENCIA_PACIENTE	CIDADE_RESIDENCIA_PACIENTE	LATI_CIDADE_RES	LONG_CIDADE_RES
A46 	430300	430300	303080078	67	3	0	0	2266474	292.62	0	20170803	20170808	24485	Rio Grande do Sul	Santa Maria	-29.6860512	-53.8069214
C168	430300	430300	304100021	60	1	0	0	2266474	797.11	0	20170803	20170805	24485	Rio Grande do Sul	Santa Maria	-29.6860512	-53.8069214
J128	430300	430300	303140151	76	3	0	1	2266474	4987.5

In [6]:
query = f"SELECT CIDADE_RESIDENCIA_PACIENTE FROM {nome_tabela} LIMIT 5;"
resultados = db.run(query)
resultados

"[('Santa Maria',), ('Santa Maria',), ('Santa Maria',), ('Santa Maria',), ('Santa Maria',)]"

# LLM Config

In [7]:
try:
    from llm_config import carregar_llm_langchain # Tenta importar a função
except ImportError as e:
    print(f"ERRO: Não foi possível importar 'carregar_llm_langchain' de llm_config.py: {e}")
    print("Certifique-se de que 'llm_config.py' está no mesmo diretório ou no PYTHONPATH.")
    # Define como None para que as próximas células possam verificar e não dar erro imediato.
    carregar_llm_langchain = None

/home/maiconkevyn/PycharmProjects/chatbot-pysus/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
llm_instance = None # Inicializa a variável

if carregar_llm_langchain:
    print("Iniciando o carregamento do LLM através do Langchain...")
    print("Este processo pode demorar um pouco, especialmente na primeira vez ou com modelos grandes.")

    # Você pode forçar o uso da CPU aqui se desejar, mudando para False,
    # mas a função em llm_config.py já tenta GPU primeiro e faz fallback para CPU se necessário.
    tentar_gpu = True

    llm_instance = carregar_llm_langchain(use_gpu_if_available=tentar_gpu)
else:
    print("A função 'carregar_llm_langchain' não foi importada. Não é possível carregar o LLM.")

if llm_instance:
    print("\n✅ LLM carregado e configurado com sucesso através do Langchain!")
else:
    print("\n❌ Falha ao carregar e configurar o LLM.")
    print("   Verifique a saída do console/terminal onde o Jupyter Notebook foi iniciado para mensagens de erro detalhadas do 'llm_config.py'.")
    # Considere interromper aqui se o LLM não carregar para evitar erros nas células seguintes.
    # raise RuntimeError("LLM não carregado. Verifique os logs.")

Iniciando o carregamento do LLM através do Langchain...
Este processo pode demorar um pouco, especialmente na primeira vez ou com modelos grandes.
Iniciando configuração do LLM via Langchain: defog/sqlcoder-7b-2
Tentando configurar para GPU com quantização de 4 bits...
CUDA parece estar funcional.
Configurado para GPU com quantização.
Carregando LLM com model_kwargs: {'trust_remote_code': True, 'quantization_config': BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
, 'torch_dtype': torch.bfloat16, 'device_map': 'auto'}
Argumento 'device' para o pipeline (se aplicável): None
Ocorreu um erro crític

In [9]:
if llm_instance:
    # ==============================================================================
    # IMPORTANTE: ESQUEMA DA TABELA 'dados_sus3'
    # ==============================================================================
    # O esquema abaixo é um EXEMPLO baseado nas colunas que você mencionou.
    # Para MELHORES RESULTADOS, substitua este pela saída EXATA do comando
    # `CREATE TABLE dados_sus3 ...` do seu banco de dados.
    # Você pode obter isso usando o seguinte código em uma célula separada ou script:
    #
    # from langchain_community.utilities import SQLDatabase
    # db_for_schema = SQLDatabase.from_uri("sqlite:///sus_data.db")
    # print(db_for_schema.get_table_info(["dados_sus3"])) # Copie o resultado aqui!
    #
    TABLE_SCHEMA = """
CREATE TABLE dados_sus3 (
    DIAG_PRINC TEXT,       -- Diagnóstico principal
    MUNIC_RES INTEGER,     -- Código do município de residência
    MUNIC_MOV INTEGER,     -- Código do município de movimentação/ocorrência
    PROC_REA TEXT,         -- Código do procedimento realizado
    IDADE INTEGER,         -- Idade do paciente
    SEXO TEXT,             -- Sexo do paciente (ex: 'MASCULINO', 'FEMININO', ou códigos)
    CID_MORTE TEXT,        -- CID da causa básica do óbito
    MORTE INTEGER,         -- Indicador de morte (ex: 1 para sim, 0 para não)
    CNES INTEGER,          -- Cadastro Nacional de Estabelecimentos de Saúde
    VAL_TOT REAL,          -- Valor total do procedimento/internação
    UTI_MES_TO INTEGER,    -- Total de dias de UTI no mês (ou indicador)
    DT_INTER TEXT,         -- Data da internação (formato YYYYMMDD ou similar como TEXT)
    DT_SAIDA TEXT,         -- Data da saída (formato YYYYMMDD ou similar como TEXT)
    total_ocorrencias INTEGER, -- Parece uma contagem pré-calculada, verifique se é relevante para queries ad-hoc
    UF_RESIDENCIA_PACIENTE TEXT, -- Sigla da UF de residência
    CIDADE_RESIDENCIA_PACIENTE TEXT, -- Nome da cidade de residência
    LATI_CIDADE_RES REAL,  -- Latitude da cidade de residência
    LONG_CIDADE_RES REAL   -- Longitude da cidade de residência
);
"""
    print("Esquema da tabela definido.")

    # Template de prompt específico para o SQLCoder
    PROMPT_TEMPLATE_SQLCODER = """### Task
    Generate a SQL query to answer the following question:
    `{question}`

    ### Database Schema
    The query will run on a database with the following schema:
    {schema}

### SQL
""" # O LLM deve gerar a query após esta linha "### SQL\n"
    print("Template do prompt definido.")

else:
    print("LLM não carregado, pulando a definição de esquema e prompt.")

LLM não carregado, pulando a definição de esquema e prompt.


In [10]:
def gerar_sql_com_llm_langchain(llm, pergunta, schema_tabela):
    """Gera SQL usando a instância LLM do Langchain."""

    prompt_completo = PROMPT_TEMPLATE.format(question=pergunta, schema=schema_tabela)

    print("Gerando SQL com LLM Langchain...")
    try:
        # Usamos o método invoke para Langchain LLMs
        resposta_llm = llm.invoke(prompt_completo)

        # Limpeza básica da saída (SQLCoder pode incluir ```sql ... ``` ou o próprio prompt)
        # A resposta do HuggingFacePipeline pode incluir o prompt original, precisamos remover.
        # Ou, se o pipeline for configurado corretamente, só a parte gerada.
        # A saída de invoke() no HuggingFacePipeline geralmente é a string completa (prompt + gerado)
        # Se o prompt estiver no início da resposta, precisamos removê-lo.
        # No entanto, a implementação padrão para text-generation já pode estar retornando só o texto gerado.
        # Se não, uma forma de pegar só o gerado:
        # if resposta_llm.startswith(prompt_completo.strip()): # Verifica se o prompt está no início
        #    generated_sql = resposta_llm[len(prompt_completo.strip()):].strip()
        # else: # Assume que é só o gerado ou precisa de outra lógica
        #    generated_sql = resposta_llm.strip()

        # O mais comum é que ele retorne apenas a completação.
        generated_sql = resposta_llm.strip()

        if "```sql" in generated_sql:
            generated_sql = generated_sql.split("```sql")[1].split("```")[0].strip()
        elif "```" in generated_sql: # Caso mais genérico de markdown block
             # Pega o que está dentro do primeiro bloco ```. Cuidado se houver múltiplos.
            parts = generated_sql.split("```")
            if len(parts) > 1:
                generated_sql = parts[1].strip() # Assume que o SQL é o primeiro bloco
                if generated_sql.startswith("sql"): # Remove o marcador 'sql' se presente
                    generated_sql = generated_sql[3:].strip()


        return generated_sql.strip()

    except Exception as e:
        print(f"Erro durante a invocação do LLM: {e}")
        return f"-- Erro ao gerar SQL: {e}"

In [11]:
if __name__ == "__main__":
    print("--- Teste de Geração Text-to-SQL com LLM carregado via Langchain ---")
    # Tenta usar GPU, com fallback para CPU se houver erro de CUDA
    llm = carregar_llm_langchain(use_gpu_if_available=True)

    if llm:
        perguntas = [
            "Quantas ocorrências de morte existem?",
            "Qual a idade média dos pacientes que tiveram morte registrada como 1?",
            "Liste as 3 UF de residência com maior número de ocorrências totais."
        ]

        for i, pergunta_natural in enumerate(perguntas):
            print(f"\n--- Pergunta {i+1} ---")
            print(f"Linguagem Natural: {pergunta_natural}")

            sql_gerado = gerar_sql_com_llm_langchain(llm, pergunta_natural, TABLE_SCHEMA)

            print("\nSQL Gerado:")
            print(sql_gerado)
            print("-" * 30)
    else:
        print("\nFalha: Não foi possível testar a geração de SQL pois o LLM Langchain não carregou.")

--- Teste de Geração Text-to-SQL com LLM carregado via Langchain ---
Iniciando configuração do LLM via Langchain: defog/sqlcoder-7b-2
Tentando configurar para GPU com quantização de 4 bits...
CUDA parece estar funcional.
Configurado para GPU com quantização.
Carregando LLM com model_kwargs: {'trust_remote_code': True, 'quantization_config': BitsAndBytesConfig {
  "_load_in_4bit": true,
  "_load_in_8bit": false,
  "bnb_4bit_compute_dtype": "bfloat16",
  "bnb_4bit_quant_storage": "uint8",
  "bnb_4bit_quant_type": "nf4",
  "bnb_4bit_use_double_quant": true,
  "llm_int8_enable_fp32_cpu_offload": false,
  "llm_int8_has_fp16_weight": false,
  "llm_int8_skip_modules": null,
  "llm_int8_threshold": 6.0,
  "load_in_4bit": true,
  "load_in_8bit": false,
  "quant_method": "bitsandbytes"
}
, 'torch_dtype': torch.bfloat16, 'device_map': 'auto'}
Argumento 'device' para o pipeline (se aplicável): None
Ocorreu um erro crítico ao carregar o LLM via Langchain: libcudart.so.11.0: cannot open shared objec